# Common DataFrame manipulations to all storage options 

In [23]:
import os
from openpyxl import load_workbook
import pandas as pd
from datetime import timedelta

BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath('')))
DATA_DIR = os.path.join(BASE_DIR, 'amerigo_island', 'data')
RAW_DATA_DIR = os.path.join(DATA_DIR, 'raw')
INTERIM_DATA_DIR = os.path.join(DATA_DIR, 'interim')

INPUT_LOAD_AND_VRE_FILENAME = 'load_and_vre.csv'
FINAL_LOAD_AND_VRE_FILENAME = 'final_load_and_vre.csv'

In [24]:
# all input columns as variables for later use
date_colname = 'date'
load_colname = 'load_mw'
solar_gen_colname = 'solar_mw'
wind_gen_colname = 'wind_mw'

load_and_vre_df = pd.read_csv(os.path.join(RAW_DATA_DIR, INPUT_LOAD_AND_VRE_FILENAME))
input_cols = [date_colname, load_colname, wind_gen_colname, solar_gen_colname]
load_and_vre_df = load_and_vre_df.iloc[:, [i for i in range(len(input_cols))]]
load_and_vre_df.columns = input_cols

In [25]:
# all derived columns as variables for later use

### COMMON - columns needed bt legacy gen, batteries, hydrogen
total_vre_gen_colname = 'total_vre_mw'
hourly_vre_gen_wo_storage_colname = 'hourly_vre_gen_wo_storage'
day_of_month_colname = 'day_of_month'
day_of_yr_colname = 'day_of_yr'
week_of_yr_colname = 'wk_of_yr'
month_colname = 'month'
year_colname = 'year'
weekday_colname = 'weekday'
load_vre_diff_colname = 'load_vre_diff'
prev_7_load_colname = 'prev_7_load'
prev_7_load_vre_diff_colname = 'prev_7_load_vre_diff'
prev_7_vre_gen_colname = 'prev_7_vre_gen'
critical_load_mw_colname = 'critical_load_mw'
critical_load_less_vre_colname = 'critical_load_less_vre_mw'
surplus_vre_colname = 'surplus_vre'
electrolyzer_power_input_mw_colname = 'electrolyzer_power_input_mw'
fuel_cell_power_generation_mw_colname = 'fuel_cell_power_generation_mw'

### BATTERIES 
cuml_load_since_prev_charge_colname = 'cuml_load_since_prev_charge_mw'
cuml_charge_since_prev_charge_colname = 'cuml_charge_since_prev_discharge_mw'
battery_soc_colname = 'battery_soc'

### HYDROGEN
req_hydrogen_kg_colname = 'reqd_hydrogen_kg'
hydrogen_production_kg_colname = 'hydrogen_prod_kg'
hydrogen_production_m3_colname = 'hydrogen_prod_m3'
hydrogen_h2o_demand_l_colname = 'hydrogen_h2o_demand_l'
hydrogen_h2o_demand_m3_colname = 'hydrogen_h2o_demand_m3'
compression_desal_demand_mw_colname = 'compression_desal_demand_mw' 
h2_demand_comp_desal_kg_colname = 'h2_demand_comp_desal_kg'
cuml_h2_storage_kg_colname = 'cuml_h2_storage_kg'
cuml_hydrogen_demand_kg_colname = 'cuml_hydrogen_demand_kg'
cuml_h2_energy_demand_mwh_colname = 'cuml_h2_energy_demand_mwh'

### BAU - LEGACY GENERATION
thermal_gen_total_mw_colname = 'thermal_gen_total_mw'
thermal_gen_rice_mw_colname = 'thermal_gen_rice_mw'
thermal_gen_combustion_mw_colname = 'thermal_gen_combustion_mw'

In [26]:
# other values/assumptions needed for the analysis
CRITICAL_LOAD_PERC = (1/3)
SOLAR_SCALE_FACTOR = 1
WIND_SCALE_FACTOR = 1

SOLAR_INSTALLATION_COST_MM = 50.8
WIND_INSTALLATION_COST_MM = 31.2

In [27]:
def clean_input_columns(df):

    cleaned_df = df.copy()
    cleaned_df = cleaned_df.dropna(how='all')

    cleaned_df[date_colname] = pd.to_datetime(cleaned_df[date_colname])
    # all solar vals need to be > 0
    cleaned_df[solar_gen_colname] = cleaned_df[solar_gen_colname]\
        .apply(lambda x: x if x > 0 else 0)
    
    cleaned_df[month_colname] = cleaned_df[date_colname].map(lambda x: x.month)
    cleaned_df[day_of_month_colname] = cleaned_df[date_colname].map(lambda x: x.day)
    cleaned_df[day_of_yr_colname] = cleaned_df[date_colname].map(lambda x: x.timetuple().tm_yday)
    
    return cleaned_df

In [28]:
cleaned_load_and_vre_df = clean_input_columns(load_and_vre_df)

In [29]:
cleaned_load_and_vre_df

,date,load_mw,wind_mw,solar_mw,month,day_of_month,day_of_yr
0,2017-01-01 00:00:00,35.1,1.67966,0.0,1,1,1
1,2017-01-01 01:00:00,35.1,1.67966,0.0,1,1,1
2,2017-01-01 02:00:00,34.6,1.23327,0.0,1,1,1
3,2017-01-01 03:00:00,34.2,1.09193,0.0,1,1,1
4,2017-01-01 04:00:00,33.8,1.46510,0.0,1,1,1
...,...,...,...,...,...,...,...
8755,2017-12-31 19:00:00,45.5,6.67250,0.0,12,31,365
8756,2017-12-31 20:00:00,46.5,6.76534,0.0,12,31,365
8757,2017-12-31 21:00:00,45.0,6.20556,0.0,12,31,365
8758,2017-12-31 22:00:00,41.7,8.72619,0.0,12,31,365


In [30]:
assert len(cleaned_load_and_vre_df) == 8760, "Expected one row per hour in year (8760), got {} rows".format(len(load_and_gen_df))

for day_of_yr in cleaned_load_and_vre_df[day_of_yr_colname].unique():
    if len(cleaned_load_and_vre_df[cleaned_load_and_vre_df[day_of_yr_colname] == day_of_yr]) != 24:
        assert ValueError("Day {} has {} values".format(day_of_yr, len(cleaned_load_and_vre_df[cleaned_load_and_vre_df[day_of_yr_colname] == day_of_yr])))
        

In [31]:
def add_common_derived_columns(df):
    
    decorated_df = df.copy()
    
    decorated_df[solar_gen_colname] = decorated_df[solar_gen_colname] * SOLAR_SCALE_FACTOR
    decorated_df[wind_gen_colname] = decorated_df[wind_gen_colname] * WIND_SCALE_FACTOR
    
    decorated_df[total_vre_gen_colname] = \
        decorated_df[wind_gen_colname] + decorated_df[solar_gen_colname]
    
    decorated_df[critical_load_mw_colname] = \
        decorated_df[load_colname] * CRITICAL_LOAD_PERC
    
    decorated_df[critical_load_less_vre_colname] = \
        decorated_df[critical_load_mw_colname] - decorated_df[total_vre_gen_colname]
    
    decorated_df[surplus_vre_colname] = \
        -decorated_df[critical_load_less_vre_colname]
    
    decorated_df[load_vre_diff_colname] = \
        decorated_df[load_colname] \
        - decorated_df[total_vre_gen_colname]
    
    decorated_df[load_vre_diff_colname] = decorated_df[load_vre_diff_colname]\
        .map(lambda x: x if x > 0 else 0)
    
    decorated_df[critical_load_less_vre_colname] = decorated_df[critical_load_less_vre_colname]\
        .map(lambda x: x if x > 0 else 0)

    decorated_df[surplus_vre_colname] = decorated_df[surplus_vre_colname]\
        .map(lambda x: x if x > 0 else 0)
    
    decorated_df[hourly_vre_gen_wo_storage_colname] = decorated_df.apply(
        lambda x: x[total_vre_gen_colname] if x[total_vre_gen_colname] <= x[critical_load_mw_colname] else x[critical_load_mw_colname],
        axis=1
    )
    

    return decorated_df

In [32]:
final_load_and_vre_df = add_common_derived_columns(cleaned_load_and_vre_df)
final_load_and_vre_df.to_csv(os.path.join(INTERIM_DATA_DIR, FINAL_LOAD_AND_VRE_FILENAME), index=False)

# Hydrogen-specific analysis

In [33]:
ELEC_EFF_H2_FUEL_CELL_KWH_PER_KG = 19.988333333333
ELEC_EFF_ELECTROLYSIS_KWH_PER_KG = 55.4307635

ENERGY_CONSUMPTION_H2O_DESAL_KWH_PER_M3 = 1.8
DENSITY_H2_300BAR_KG_PER_M3 = 20
DENSITY_H2_30BAR_KG_PER_M3 = 2.38
H2O_ELECTROLYSIS_CONSUMPTION_L_PER_M3 = 1.4

DENSITY_H2_STP_KG_PER_M3 = 0.0813
H2_TAP_WATER_CONSUMPTION_L_PER_M3 = 1.4

ENERGY_CONSUMPTION_H2_KWH_PER_KG = 4
STARTING_H2_IN_STORAGE_KG = 10000

def calc_h2_cols(df):
    
    decorated_df = df.copy()
    
    decorated_df[req_hydrogen_kg_colname] = \
        (decorated_df[critical_load_less_vre_colname] * 1000)/ ELEC_EFF_H2_FUEL_CELL_KWH_PER_KG
    
    decorated_df[hydrogen_production_kg_colname] = \
        (decorated_df[surplus_vre_colname] * 1000) / ELEC_EFF_ELECTROLYSIS_KWH_PER_KG
    
    decorated_df[hydrogen_production_m3_colname] = \
        decorated_df[hydrogen_production_kg_colname] / DENSITY_H2_30BAR_KG_PER_M3
    
    decorated_df[hydrogen_h2o_demand_l_colname] = \
        decorated_df[hydrogen_production_m3_colname] * H2O_ELECTROLYSIS_CONSUMPTION_L_PER_M3
    
    decorated_df[hydrogen_h2o_demand_m3_colname] = \
        (decorated_df[hydrogen_production_kg_colname] / DENSITY_H2_STP_KG_PER_M3) \
        * (H2_TAP_WATER_CONSUMPTION_L_PER_M3 / 1000)
    
    decorated_df[compression_desal_demand_mw_colname] = \
        (decorated_df[hydrogen_production_kg_colname] * ENERGY_CONSUMPTION_H2_KWH_PER_KG / 1000) \
        + (decorated_df[hydrogen_h2o_demand_m3_colname] * ENERGY_CONSUMPTION_H2O_DESAL_KWH_PER_M3 / 1000)
    
    decorated_df[h2_demand_comp_desal_kg_colname] = \
        (decorated_df[compression_desal_demand_mw_colname] * 1000)/ ELEC_EFF_H2_FUEL_CELL_KWH_PER_KG
    
    decorated_df.sort_values(date_colname)
    
# calc cuml h2 demand
    decorated_df[cuml_hydrogen_demand_kg_colname] = None
    decorated_df[cuml_h2_storage_kg_colname] = None
    decorated_df.reset_index()
    for idx, row in decorated_df.iterrows():
        
        if idx == 0:
            h2_demand_val = row[req_hydrogen_kg_colname]
            h2_storage_val = STARTING_H2_IN_STORAGE_KG \
                - row[req_hydrogen_kg_colname] \
                - row[h2_demand_comp_desal_kg_colname] \
                + row[hydrogen_production_kg_colname]

            
        else : #
            
            h2_demand_val = \
                decorated_df.loc[idx - 1, cuml_hydrogen_demand_kg_colname] \
                + row[req_hydrogen_kg_colname]\
                - row[hydrogen_production_kg_colname]
            
            h2_storage_val = decorated_df.loc[idx - 1, cuml_h2_storage_kg_colname] \
                - row[req_hydrogen_kg_colname] \
                - row[h2_demand_comp_desal_kg_colname] \
                + row[hydrogen_production_kg_colname]
        
        decorated_df.at[idx, cuml_hydrogen_demand_kg_colname] = h2_demand_val
        decorated_df.at[idx, cuml_h2_storage_kg_colname] = h2_storage_val
    
        
    decorated_df[electrolyzer_power_input_mw_colname] = \
        decorated_df[hydrogen_production_kg_colname] * ELEC_EFF_ELECTROLYSIS_KWH_PER_KG/ 1000
    decorated_df[cuml_h2_energy_demand_mwh_colname] = \
        (decorated_df[cuml_hydrogen_demand_kg_colname] * ELEC_EFF_H2_FUEL_CELL_KWH_PER_KG) / 1000
    decorated_df[fuel_cell_power_generation_mw_colname] = \
        (decorated_df[req_hydrogen_kg_colname] \
        + decorated_df[h2_demand_comp_desal_kg_colname]) \
        * (ELEC_EFF_H2_FUEL_CELL_KWH_PER_KG / 1000)
    

    return decorated_df

In [34]:
h2_hourly_df = calc_h2_cols(final_load_and_vre_df)

h2_daily_df = h2_hourly_df.groupby(day_of_yr_colname).sum().reset_index()
h2_daily_df = h2_daily_df.drop([month_colname, day_of_month_colname], axis=1)

In [35]:
h2_daily_df

,day_of_yr,load_mw,wind_mw,solar_mw,total_vre_mw,critical_load_mw,critical_load_less_vre_mw,surplus_vre,load_vre_diff,hourly_vre_gen_wo_storage,reqd_hydrogen_kg,hydrogen_prod_kg,hydrogen_prod_m3,hydrogen_h2o_demand_l,hydrogen_h2o_demand_m3,compression_desal_demand_mw,h2_demand_comp_desal_kg,electrolyzer_power_input_mw,fuel_cell_power_generation_mw
0,1,949.3,33.731947,133.287,167.018947,316.433333,172.343951,22.929565,782.281053,144.089382,8622.227196,413.661366,173.807297,243.330215,7.123320,1.667467,83.422035,22.929565,174.011419
1,2,1014.8,128.402210,154.156,282.558210,338.266667,124.958597,69.250140,732.241790,213.308070,6251.576586,1249.308789,524.919659,734.887523,21.513312,5.035959,251.944924,69.250140,129.994556
2,3,1019.5,87.408180,103.099,190.507180,339.833333,167.123620,17.797467,828.992820,172.709713,8361.058284,321.075618,134.905722,188.868011,5.528977,1.294255,64.750503,17.797467,168.417875
3,4,1007.8,108.888920,124.489,233.377920,335.933333,135.719780,33.164367,774.422080,200.213553,6789.949804,598.302541,251.387622,351.942671,10.302873,2.411755,120.658151,33.164367,138.131535
4,5,1001.6,75.726760,114.990,190.716760,333.866667,160.873510,17.723603,810.883240,172.993157,8048.370383,319.743085,134.345834,188.084168,5.506031,1.288883,64.481774,17.723603,162.162393
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,361,929.3,3.726741,141.770,145.496741,309.766667,191.677936,27.408011,783.803259,118.088731,9589.490661,494.454865,207.754145,290.855803,8.514598,1.993146,99.715454,27.408011,193.671082
361,362,916.9,30.535031,153.303,183.838031,305.633333,174.263719,52.468417,733.061969,131.369614,8718.271608,946.557712,397.713324,556.798654,16.299887,3.815571,190.889885,52.468417,178.079290
362,363,953.5,12.596823,136.703,149.299823,317.833333,196.688154,28.154644,804.200177,121.145179,9840.147791,507.924515,213.413662,298.779126,8.746548,2.047442,102.431844,28.154644,198.735596
363,364,906.5,63.291317,138.074,201.365317,302.166667,144.256400,43.455050,705.134683,157.910267,7217.029917,783.951857,329.391537,461.148151,13.499786,3.160107,158.097576,43.455050,147.416507


In [36]:
h2_hourly_df[h2_hourly_df[day_of_yr_colname] == 361].to_csv('./day361.csv', index=False)

# Add common columns to daily DataFrame

In [43]:
def sum_prev_rows(row, df, lookback_colname, agg_colname, days = 7):

    start_idx = row[lookback_colname] - days    
    return_val = df.loc[start_idx:row[lookback_colname] - 1, agg_colname].sum() if start_idx >= 0 else None
    
    return return_val

def add_daily_common_cols(df):
    
    decorated_df = df.copy()

    decorated_df[prev_7_load_colname] = decorated_df\
        .apply(sum_prev_rows, axis=1, args=(decorated_df, day_of_yr_colname, load_colname,))

    decorated_df[prev_7_load_vre_diff_colname] = decorated_df\
        .apply(sum_prev_rows, axis=1, args=(decorated_df, day_of_yr_colname, load_vre_diff_colname,))

    decorated_df[prev_7_vre_gen_colname] = decorated_df\
        .apply(sum_prev_rows, axis=1, args=(decorated_df, day_of_yr_colname, total_vre_gen_colname,))

    return decorated_df

In [44]:
daily_df = add_daily_common_cols(h2_daily_df)

# Battery-specific analysis

In [87]:
BATTERY_CAPACITY_MWH = 350
BATTERY_POWER_MW = 29
THERMAL_BASE_GEN_MW = 31

battery_soc_colname = 'battery_soc'
battery_curtailment_colname = 'battery_curtailment'
load_after_thermal_base_gen_mw_colname = 'load_after_thermal_base_gen_mw'
remaining_load_after_vre_prod_mw_colname = 'remaining_load_after_vre_prod_mw'

battery_capacity_mwh_config_prop = "battery_capacity_mwh"
battery_power_mw_config_prop = "battery_power_mw"

def add_battery_cols(hourly_df, battery_config):
    
    decorated_df = hourly_df.copy()
    
    battery_capacity = battery_config[battery_capacity_mwh_config_prop]

    decorated_df[load_after_thermal_base_gen_mw_colname] = \
        decorated_df[load_colname] - THERMAL_BASE_GEN_MW
        
    decorated_df[remaining_load_after_vre_prod_mw_colname] = \
        decorated_df[load_after_thermal_base_gen_mw_colname] \
        - decorated_df[total_vre_gen_colname]
    
    decorated_df[remaining_load_after_vre_prod_mw_colname] = \
        decorated_df[remaining_load_after_vre_prod_mw_colname].map(lambda x: x if x > 0 else 0)
    
    decorated_df[battery_soc_colname] = None
    decorated_df[battery_curtailment_colname] = None
    
    decorated_df.reset_index()
    for idx, row in decorated_df.iterrows():

        if idx == 0:
            
            soc_val = \
                battery_capacity \
                - row[remaining_load_after_vre_prod_mw_colname] \
                + row[surplus_vre_colname]
            
            batt_curtailment_val = 0
                
        else:
            soc_val = \
                decorated_df.loc[idx - 1, battery_soc_colname]\
                - row[remaining_load_after_vre_prod_mw_colname]\
                + row[surplus_vre_colname]
            soc_val = soc_val if soc_val < battery_capacity else battery_capacity
            soc_val = soc_val if soc_val > 0 else 0
            
            batt_curtailment_val = \
                decorated_df.loc[idx - 1, battery_soc_colname] \
                - row[remaining_load_after_vre_prod_mw_colname]\
                + row[surplus_vre_colname]\
                - soc_val
            
            batt_curtailment_val = batt_curtailment_val if batt_curtailment_val > 0 else 0

        
        decorated_df.at[idx, battery_soc_colname] = soc_val
        decorated_df.at[idx, battery_curtailment_colname] = batt_curtailment_val

    decorated_df[battery_soc_colname] = decorated_df[battery_soc_colname].astype(float)
    decorated_df[battery_curtailment_colname] = decorated_df[battery_curtailment_colname].astype(float)
    return decorated_df

In [88]:
battery_config = {
    battery_capacity_mwh_config_prop : BATTERY_CAPACITY_MWH,
    battery_power_mw_config_prop : BATTERY_POWER_MW
}

In [89]:
battery_hourly_df = add_battery_cols(h2_hourly_df, battery_config)

In [90]:
battery_hourly_df[[
    battery_soc_colname, 
    battery_curtailment_colname,
]].describe()

,battery_soc,battery_curtailment
count,8760.000000,8760.000000
mean,118.487987,0.446168
std,127.396333,2.450021
min,0.000000,0.000000
25%,0.687618,0.000000
50%,60.509180,0.000000
75%,240.811689,0.000000
max,350.000000,22.667067


In [91]:
battery_hourly_df

,date,load_mw,wind_mw,solar_mw,month,day_of_month,day_of_yr,total_vre_mw,critical_load_mw,critical_load_less_vre_mw,...,h2_demand_comp_desal_kg,cuml_hydrogen_demand_kg,cuml_h2_storage_kg,electrolyzer_power_input_mw,cuml_h2_energy_demand_mwh,fuel_cell_power_generation_mw,load_after_thermal_base_gen_mw,remaining_load_after_vre_prod_mw,battery_soc,battery_curtailment
0,2017-01-01 00:00:00,35.1,1.67966,0.0,1,1,1,1.67966,11.700000,10.020340,...,0.0,501.309,9498.69,0.0,10.0203,10.020340,4.1,2.42034,347.57966,0.0
1,2017-01-01 01:00:00,35.1,1.67966,0.0,1,1,1,1.67966,11.700000,10.020340,...,0.0,1002.62,8997.38,0.0,20.0407,10.020340,4.1,2.42034,345.15932,0.0
2,2017-01-01 02:00:00,34.6,1.23327,0.0,1,1,1,1.23327,11.533333,10.300063,...,0.0,1517.92,8482.08,0.0,30.3407,10.300063,3.6,2.36673,342.79259,0.0
3,2017-01-01 03:00:00,34.2,1.09193,0.0,1,1,1,1.09193,11.400000,10.308070,...,0.0,2033.63,7966.37,0.0,40.6488,10.308070,3.2,2.10807,340.68452,0.0
4,2017-01-01 04:00:00,33.8,1.46510,0.0,1,1,1,1.46510,11.266667,9.801567,...,0.0,2523.99,7476.01,0.0,50.4504,9.801567,2.8,1.33490,339.34962,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2017-12-31 19:00:00,45.5,6.67250,0.0,12,31,365,6.67250,15.166667,8.494167,...,0.0,1.91219e+06,-1.97962e+06,0.0,38221.5,8.494167,14.5,7.82750,151.77412,0.0
8756,2017-12-31 20:00:00,46.5,6.76534,0.0,12,31,365,6.76534,15.500000,8.734660,...,0.0,1.91263e+06,-1.98006e+06,0.0,38230.2,8.734660,15.5,8.73466,143.03946,0.0
8757,2017-12-31 21:00:00,45.0,6.20556,0.0,12,31,365,6.20556,15.000000,8.794440,...,0.0,1.91307e+06,-1.9805e+06,0.0,38239,8.794440,14.0,7.79444,135.24502,0.0
8758,2017-12-31 22:00:00,41.7,8.72619,0.0,12,31,365,8.72619,13.900000,5.173810,...,0.0,1.91333e+06,-1.98076e+06,0.0,38244.2,5.173810,10.7,1.97381,133.27121,0.0


# Scratch
##### Unsure if needed. Keeping just in case

In [47]:
resilience_case_load_daynum = \
    daily_df[daily_df[prev_7_load_colname] == daily_df[prev_7_load_colname].max()].to_dict('r')[0][day_of_yr_colname]

resilience_case_vre_load_diff_daynum = \
    daily_df[daily_df[prev_7_load_vre_diff_colname] == daily_df[prev_7_load_vre_diff_colname].max()].to_dict('r')[0][day_of_yr_colname]

resilience_case_max_demand_daynum = \
    final_load_and_vre_df[final_load_and_vre_df[load_colname] == final_load_and_vre_df[load_colname].max()].to_dict('r')[0][day_of_yr_colname]


In [51]:
def add_storage_input_cols(df, daynum):
    
    decorated_df = df.copy()
    
    decorated_df = decorated_df[
        (decorated_df[day_of_yr_colname] <= daynum)
        & (decorated_df[day_of_yr_colname] >= daynum - 6)
    ]
    
    decorated_df = decorated_df.reset_index(drop=True)
    
    decorated_df.sort_values(date_colname)
    decorated_df[cuml_load_since_prev_charge_colname] = None
    
    for idx, row in decorated_df.iterrows():
        
        if idx == 0:
            val = row[critical_load_less_vre_colname]
        elif row[critical_load_less_vre_colname] > 0:
            val = decorated_df.loc[idx - 1, cuml_load_since_prev_charge_colname] + row[critical_load_less_vre_colname]
        else:
            val = 0
            
        decorated_df.at[idx, cuml_load_since_prev_charge_colname] = val
        
    return decorated_df

In [52]:
resil_max_week_load_df = add_storage_input_cols(h2_hourly_df, resilience_case_load_daynum)
resil_vre_load_diff_df = add_storage_input_cols(h2_hourly_df, resilience_case_vre_load_diff_daynum)
resil_max_demand_df = add_storage_input_cols(h2_hourly_df, resilience_case_max_demand_daynum)

In [54]:
resil_max_week_load_df

,date,load_mw,wind_mw,solar_mw,month,day_of_month,day_of_yr,total_vre_mw,critical_load_mw,critical_load_less_vre_mw,...,hydrogen_h2o_demand_l,hydrogen_h2o_demand_m3,compression_desal_demand_mw,h2_demand_comp_desal_kg,cuml_hydrogen_demand_kg,cuml_h2_storage_kg,electrolyzer_power_input_mw,cuml_h2_energy_demand_mwh,fuel_cell_power_generation_mw,cuml_load_since_prev_charge_mw
0,2017-08-14 00:00:00,40.8,3.391490,0.000,8,14,226,3.391490,13.600000,10.208510,...,0.0,0.0,0.0,0.0,1.02197e+06,-1.06603e+06,0.0,20427.5,10.208510,10.2085
1,2017-08-14 01:00:00,39.2,4.369580,0.000,8,14,226,4.369580,13.066667,8.697087,...,0.0,0.0,0.0,0.0,1.02241e+06,-1.06646e+06,0.0,20436.2,8.697087,18.9056
2,2017-08-14 02:00:00,38.3,4.737650,0.000,8,14,226,4.737650,12.766667,8.029017,...,0.0,0.0,0.0,0.0,1.02281e+06,-1.06686e+06,0.0,20444.3,8.029017,26.9346
3,2017-08-14 03:00:00,37.5,5.029210,0.000,8,14,226,5.029210,12.500000,7.470790,...,0.0,0.0,0.0,0.0,1.02318e+06,-1.06724e+06,0.0,20451.7,7.470790,34.4054
4,2017-08-14 04:00:00,37.5,4.012080,0.000,8,14,226,4.012080,12.500000,8.487920,...,0.0,0.0,0.0,0.0,1.02361e+06,-1.06766e+06,0.0,20460.2,8.487920,42.8933
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,2017-08-20 19:00:00,50.3,0.886276,0.054,8,20,232,0.940276,16.766667,15.826391,...,0.0,0.0,0.0,0.0,1.05694e+06,-1.10231e+06,0.0,21126.4,15.826391,253.142
164,2017-08-20 20:00:00,50.2,0.699870,0.000,8,20,232,0.699870,16.733333,16.033463,...,0.0,0.0,0.0,0.0,1.05774e+06,-1.10311e+06,0.0,21142.4,16.033463,269.175
165,2017-08-20 21:00:00,48.5,1.539350,0.000,8,20,232,1.539350,16.166667,14.627317,...,0.0,0.0,0.0,0.0,1.05847e+06,-1.10384e+06,0.0,21157.1,14.627317,283.802
166,2017-08-20 22:00:00,46.6,0.829267,0.000,8,20,232,0.829267,15.533333,14.704066,...,0.0,0.0,0.0,0.0,1.05921e+06,-1.10458e+06,0.0,21171.8,14.704066,298.506


In [53]:
def calc_soc_cols(df, config):
    
    decorated_df = df.copy()
    batt_energy_mwh = config[batt_energy_mwh_keyname]
    
    decorated_df[battery_soc_colname] = None
    decorated_df.reset_index(drop=True)
    
    for idx, row in decorated_df.iterrows():
        
        if idx == 0:
            val = batt_energy_mwh - row[critical_load_less_vre_colname] + row[charge_surplus_colname]
        else:
            charge = decorated_df.loc[idx - 1, battery_soc_colname] - row[critical_load_less_vre_colname] + row[charge_surplus_colname] 
            val = charge if charge < batt_energy_mwh else batt_energy_mwh
            
        decorated_df.at[idx, battery_soc_colname] = val
    
    return decorated_df

,date,load_mw,wind_mw,solar_mw,month,day_of_month,day_of_yr,total_vre_mw,critical_load_mw,critical_load_less_vre_mw,...,hydrogen_h2o_demand_l,hydrogen_h2o_demand_m3,compression_desal_demand_mw,h2_demand_comp_desal_kg,cuml_hydrogen_demand_kg,cuml_h2_storage_kg,electrolyzer_power_input_mw,cuml_h2_energy_demand_mwh,fuel_cell_power_generation_mw,cuml_load_since_prev_charge_mw
0,2017-08-14 00:00:00,40.8,3.391490,0.000,8,14,226,3.391490,13.600000,10.208510,...,0.0,0.0,0.0,0.0,1.02197e+06,-1.06603e+06,0.0,20427.5,10.208510,10.2085
1,2017-08-14 01:00:00,39.2,4.369580,0.000,8,14,226,4.369580,13.066667,8.697087,...,0.0,0.0,0.0,0.0,1.02241e+06,-1.06646e+06,0.0,20436.2,8.697087,18.9056
2,2017-08-14 02:00:00,38.3,4.737650,0.000,8,14,226,4.737650,12.766667,8.029017,...,0.0,0.0,0.0,0.0,1.02281e+06,-1.06686e+06,0.0,20444.3,8.029017,26.9346
3,2017-08-14 03:00:00,37.5,5.029210,0.000,8,14,226,5.029210,12.500000,7.470790,...,0.0,0.0,0.0,0.0,1.02318e+06,-1.06724e+06,0.0,20451.7,7.470790,34.4054
4,2017-08-14 04:00:00,37.5,4.012080,0.000,8,14,226,4.012080,12.500000,8.487920,...,0.0,0.0,0.0,0.0,1.02361e+06,-1.06766e+06,0.0,20460.2,8.487920,42.8933
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,2017-08-20 19:00:00,50.3,0.886276,0.054,8,20,232,0.940276,16.766667,15.826391,...,0.0,0.0,0.0,0.0,1.05694e+06,-1.10231e+06,0.0,21126.4,15.826391,253.142
164,2017-08-20 20:00:00,50.2,0.699870,0.000,8,20,232,0.699870,16.733333,16.033463,...,0.0,0.0,0.0,0.0,1.05774e+06,-1.10311e+06,0.0,21142.4,16.033463,269.175
165,2017-08-20 21:00:00,48.5,1.539350,0.000,8,20,232,1.539350,16.166667,14.627317,...,0.0,0.0,0.0,0.0,1.05847e+06,-1.10384e+06,0.0,21157.1,14.627317,283.802
166,2017-08-20 22:00:00,46.6,0.829267,0.000,8,20,232,0.829267,15.533333,14.704066,...,0.0,0.0,0.0,0.0,1.05921e+06,-1.10458e+06,0.0,21171.8,14.704066,298.506
